In [2]:
import numpy as np
import tensorflow as tf
import cv2
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer


In [3]:
print(tf.__version__)

2.13.0


In [4]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability = None
)

print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Num GPUs Available:  []


In [5]:
#Image Preprocessor class

class ImagePreprocessor:
    def __init__(self, root_dir, desired_size=(128, 128)):
        self.root_dir = root_dir
        self.desired_size = desired_size
        ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

        self.datagen = ImageDataGenerator(
            rotation_range=20,
            zoom_range=0.15,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.15,
            horizontal_flip=True,
            fill_mode="nearest"
        )
        
    def _load_image(self, path):
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        image = cv2.resize(image, self.desired_size)
        return image
    
    def process_images(self):
        images = []
        labels = []

        for label in os.listdir(self.root_dir):
            class_path = os.path.join(self.root_dir, label)
            if os.path.isdir(class_path):
                for filename in os.listdir(class_path):
                    filepath = os.path.join(class_path, filename)
                    if filepath.lower().endswith(('.jpg', '.jpeg')):
                        image = self._load_image(filepath)
                        images.append(image)
                        labels.append(label)
        
        # Convert images and labels to arrays
        images = np.array(images) / 255.0
        labels = np.array(labels)
        print("Unique Labels: ", np.unique(labels))

        # One-hot encode labels
        lb = LabelBinarizer()
        labels = lb.fit_transform(labels)
        return images, labels

    def get_train_val_test(self, test_size=0.2, val_size=0.1):
        images, labels = self.process_images()
        trainX, testX, trainY, testY = train_test_split(images, labels, test_size=test_size)
        trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=val_size / (1 - test_size))
        
        return (trainX, trainY), (valX, valY), (testX, testY)

    def save_processed_images(self, save_dir):

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        images, labels = self.process_images()

        #Save images as file name

        for idx, (image, label) in enumerate(zip(images, labels)):
            label_str = str(np.argmax(label))
            filename = f"{label_str}_{idx}.jpg"
            filepath = os.path.join(save_dir, filename)

            # Convert the image back to the range [0, 255]
            image_to_save = (image * 255).astype(np.uint8)

            cv2.imwrite(filepath, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


In [6]:
#Load MobileNetV2 without top layer

base_model = MobileNetV2(weights='imagenet', include_top=False)

In [7]:
#Average Pooling Layer

x = base_model.output
x = GlobalAveragePooling2D()(x)

#fully-connected layer 

x = Dense(1024, activation = 'relu')(x)

# softmax layer for classification

num_classes = len(os.listdir("C:\\Users\\bcurl\\Desktop\\AnimalDetect\\data\\raw-img")) 

predictions = Dense(num_classes, activation='softmax')(x)

#trained models

model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
#Freeze pre trained layers

for layer in base_model.layers:
    layer.trainable = False
    

In [9]:
#Compile the model 

model.compile(optimizer = 'adam', loss='categorical_crossentropy',metrics =['accuracy'])

In [10]:
#process images 

preprocessor = ImagePreprocessor("C:\\Users\\bcurl\\Desktop\\AnimalDetect\\data\\raw-img")
(trainX, trainY), (valX, valY), (testX, testY) = preprocessor.get_train_val_test()

Unique Labels:  ['butterfly' 'cat' 'chicken' 'cow' 'dog' 'elephant' 'horse' 'sheep'
 'spider' 'squirrel']


In [11]:
#Train the model with the new layers for some epochs

model.fit(trainX, trainY, validation_data=(valX, valY), epochs=10, batch_size=32)

Epoch 1/10
572/572 [==============================] - 98s 166ms/step - loss: 0.4325 - accuracy: 0.8704 - val_loss: 0.3707 - val_accuracy: 0.8775
Epoch 2/10
572/572 [==============================] - 92s 161ms/step - loss: 0.2372 - accuracy: 0.9208 - val_loss: 0.3262 - val_accuracy: 0.8990
Epoch 3/10
572/572 [==============================] - 91s 159ms/step - loss: 0.1730 - accuracy: 0.9419 - val_loss: 0.3692 - val_accuracy: 0.8860
Epoch 4/10
572/572 [==============================] - 91s 159ms/step - loss: 0.1205 - accuracy: 0.9586 - val_loss: 0.3616 - val_accuracy: 0.8948
Epoch 5/10
572/572 [==============================] - 91s 160ms/step - loss: 0.0854 - accuracy: 0.9704 - val_loss: 0.4420 - val_accuracy: 0.8814
Epoch 6/10
572/572 [==============================] - 90s 157ms/step - loss: 0.0690 - accuracy: 0.9764 - val_loss: 0.4120 - val_accuracy: 0.8982
Epoch 7/10
572/572 [==============================] - 91s 159ms/step - loss: 0.0558 - accuracy: 0.9814 - val_loss: 0.4811 - val_ac

In [13]:
model.save("animal_detection_model_1.keras")

In [14]:
loss, accuracy = model.evaluate(testX, testY)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


164/164 [==============================] - 22s 137ms/step - loss: 0.6399 - accuracy: 0.8986
Test Loss: 0.6398894786834717
Test Accuracy: 0.8985840082168579
